In [1]:
import math as math 
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

#facebook data
#file_name='/Users/ssingson-robbins/Desktop/Grad School/facebook_combined.txt'
dat=np.genfromtxt(file_name,dtype=str)


In [56]:
# First, Vertex and Graph classes for directed graphs

class Vertex:
    def __init__(self, v):
        self.Neighbors = []
        self.Edges = []
        self.value = v
        # useful for DFS
        self.community = 0
        self.status = "unvisited"
        self.score = 0 
        self.audiences = {}
        
    def hasNeighbor(self,u):
        if u in self.Neighbors:
            return True
        return False
        
    def getNeighbors(self):
        return self.Neighbors
        
    def addNeighbor(self,u):
        self.Neighbors.append(u)
        
    def removeNeighbor(self,u): 
        self.Neighbors.remove(u)
        
    def edgeNeighbors(self, edge): 
        neighbors = []
        for vertex in edge.Vertex: 
            for edges in vertex: 
                neighbors.append(edges)
        return neighbors
    
    def addAudience(self, u, v): 
        self.audience[u] = v
    
    def __str__(self):
        return str(self.value) 

class Edge:
    def __init__(self, vertexes): 
        self.Vertex = vertexes
        self.status = 'unvisited'
        self.score = 0
        self.roundscore = 0
        self.finalscore = 0
    
    def getVertices(self): 
        return self.Vertex
    
    def edgeNeighbor(self): 
        neighbors = []
        for vertex in self.Vertex: 
            for edge in vertex.Edges: 
                if edge != self: 
                    neighbors.append(edge)
        return neighbors

    def secondVertex(self, v): 
        for i in self.Vertex: 
            if v != i:
                return i
        
    def __str__(self):
        return "(" + str(self.Vertex[0]) + ", " + str(self.Vertex[1]) + ")"
        
class Graph:
    def __init__(self):
        self.Vertices = []
        self.Edges = []
        self.status = 'unvisited'

    def addVertex(self,n):
        self.Vertices.append(n)
        
    # add a directed edge from Node u to Node v
    def addEdge(self,u,v):
        u.addNeighbor(v)
        v.addNeighbor(u)
        new_edge = Edge((u,v))
        self.Edges.append(new_edge)
        u.Edges.append(new_edge)
        v.Edges.append(new_edge)
        
    def removeEdge(self, i): 
        first_vertex = i.Vertex[0]
        second_vertex = i.Vertex[1]
        first_vertex.Neighbors.remove(second_vertex)
        second_vertex.Neighbors.remove(first_vertex)
        first_vertex.Edges.remove(i)
        second_vertex.Edges.remove(i)
        self.Edges.remove(i)
        
    # get a list of all the directed edges
    # directed edges are a list of two Vertices
    def getEdges(self):
        ret = ""
        for v in self.Edges:
            ret += str(v) + ","
        return ret
    
    def vertexEdges(self, v): 
        neighbors = []
        for i in self.Edges:
            if v.edgeNeighbor(i) == True: 
                neighbors.append(i)
        return neighbors
    
    def __str__(self):
        ret = "Graph with:\n"
        ret += "\t Vertices:\n\t"
        for v in self.Vertices:
            ret += str(v) + ","
        ret += "\n"
        ret += "\t Edges:\n\t"
        for v in self.Edges:
            ret += str(v) + ","
        ret += "\n"
        return ret


In [58]:
#Breadth First Search to find the shortest paths to all the nodes 
def BFS(w, G, community = 0):
    n = len(G.Vertices)
    Ls = [ [] for i in range(n) ]
    Ls[0] = [w]
    w.status = "visited"
    w.community = community 
    for i in range(n):
        for u in Ls[i]:
            for v in u.getNeighbors():
                if v.status == "unvisited":
                    v.community = community #Used to save the vertex's community 
                    v.status = "visited"
                    Ls[i+1].append(v)
    return Ls

#Scores the nodes for community detection
def nodeScore(w,G, Ls):
    lastEdge = 0
    for v in G.Vertices:
        v.score = 0
        v.status = "unvisited"
    w.score = 1
    # okay, now what are all the shortest paths?
    for i in range(len(Ls)):
        for w in Ls[i]:
            for j in w.getNeighbors(): 
                if j.status == 'visited':
                    w.score += j.score
            w.status = 'visited'
            lastEdge = w
    return lastEdge

#Scores the edges for community detection
def edgeScore(w, G): 
    for v in G.Vertices:
        v.status = "unvisited"
    Ls = BFS(w,G)
    first_node = nodeScore(w,G, Ls)
    for i in G.Vertices: 
        i.status = 'unvisited'
    for i in G.Edges:
        i.roundscore = 0
        i.status = 'unvisited'
    for i in first_node.Edges:
        i.roundscore = i.secondVertex(first_node).score / first_node.score
        i.status = 'visited'
    for paths in range(len(Ls)-1, -1, -1):
        for vertex in Ls[paths]:            
            if vertex == first_node:
                break
            for edge in vertex.Edges: 
                if edge.status == 'unvisited': 
                    for neighbor in edge.edgeNeighbor(): 
                        if neighbor.status == 'visited' and vertex in neighbor.Vertex: 
                            edge.roundscore += neighbor.roundscore

                    edge.roundscore = (1 + edge.roundscore) / vertex.score
            for edge in vertex.Edges: 
                edge.status = 'visited'
    for i in G.Edges: 
        i.finalscore += i.roundscore

#Final scores to determine which edges to keep and remove
def allEdgesScore(G): 
    for edge in G.Edges: 
        edge.finalscore = 0
    for vertex in G.Vertices: #for every edge, runs the node and edge score to sum up the final score
        edgeScore(vertex, G)

In [60]:
#FACEBOOK DATA
# V = set()
# for i, j in dat:
#     V.add(int(i))
#     V.add(int(j))
# V = sorted(V)

# G = Graph()

# for i in V: 
#     G.addVertex(Vertex(i))
# V = G.Vertices
# for i, j in dat: 
#     G.addEdge(V[int(i)],V[int(j)])
# print('First Part Done')
# allEdgesScore(G)
# for i in G.Edges:
#     print(i)
#     print(i.finalscore)

In [86]:
V = {'A','B','C','D','E','F'}

H = Graph()


H.addVertex(Vertex('A'))
H.addVertex(Vertex('B'))
H.addVertex(Vertex('C'))
H.addVertex(Vertex('D'))
H.addVertex(Vertex('E'))
H.addVertex(Vertex('F'))

V = H.Vertices

H.addEdge(V[0], V[1])
H.addEdge(V[1], V[2])
H.addEdge(V[3], V[4])
H.addEdge(V[4], V[5])
H.addEdge(V[0], V[3])
H.addEdge(V[3], V[0])
H.addEdge(V[1], V[4])
H.addEdge(V[2], V[5])

['A', 'B', 'C', 'D', 'E', 'F']

In [100]:
# Table Version of the data 
graph_example_data = {
    'Node': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Edges': [
        ['B', 'D'],
        ['C', 'E'],
        ['F'],
        ['E'],
        ['F'], 
        []
    ]
}

# Create the DataFrame
graph_example_data = pd.DataFrame(graph_example_data).sort_values(by='Node')

def csv_create_function(data):
    new_graph = Graph()
    #Creates a graph from a csv file. 
    #The file should have a column called 'nodes' and a comma delimited list of nodes. 
    for Node in graph_example_data['Node']: 
        new_graph.addVertex(Vertex(Node))

    vertice_objects = new_graph.Vertices
    vertice_names = [vertice.value for vertice in vertice_objects]

    for Edge in range(len(data)):
        for j in range(len(data.loc[i]['Edges'])): 
            vertex1 = data.loc[i]['Node']
            vertex2 = data.loc[i]['Edges'][j]
            if vertex1 < vertex2: 
                new_graph.addEdge(vertice_objects[get_index(vertex1, vertice_names)], vertice_objects[get_index(vertex2, vertice_names)])
    
    return new_graph

csv_create_function(graph_example_data) 

A B
A D
B C
B E
C F
D E
E F


In [15]:
def community_count(H):
    communities = 0
    for i in H.Vertices:
        i.status = 'unvisited'
    for i in H.Vertices: 
        if i.status == 'unvisited':
            communities += 1
            BFS(i, H, communities)
    return communities

#remove edges until graph separated into specific number of communities
def community_divider(G, num_communities):
    allEdgesScore(G)
    sorted_edges = G.Edges.sort(key=lambda x: x.finalscore, reverse=True)
    while community_count(G) < num_communities:
        H.removeEdge(H.Edges[0])

community_divider(H, 2)
for i in range(len(H.Edges)):
    print('Edge ' + str(H.Edges[i]) + ": " + str(H.Edges[i].finalscore))
for i in H.Vertices:
    print(i, i.community)



Edge (A, B): 7.999999999999999
Edge (D, E): 7.999999999999999
Edge (B, E): 7.333333333333332
Edge (A, D): 5.333333333333333
Edge (C, F): 5.333333333333333
A 1
B 1
C 2
D 1
E 1
F 2


In [17]:
def community_size(community_list): 
    communities = {}
    for i in community_list:
        if i.community not in communities: 
            communities[i.community] = 1
        else: 
            communities[i.community] += 1
    return communities

def community_vertex_list(community_list): 
    communities = {}
    for i in community_list:
        if i.community not in communities: 
            communities[i.community] = [i]
        else: 
            communities[i.community].append(i)            
    return communities
        
community_size(H.Vertices)

{1: 4, 2: 2}

In [29]:
user_list = ['A', 'B', 'C']

def userlist_to_objects(user_list, Graph): 
    object_list = []
    for i in user_list: 
        object_list.append([vertices for vertices in Graph.Vertices if vertices.value == i][0])
    return object_list

def lookalike_audience(user_list, H, new_users, user_score = 0.5, network_score = 0.5, combined_score = 0): 
    lookalike_list = [] #list to save the list of users 
    relevance_score = {} 
    user_counts = community_size(userlist_to_objects(user_list, H))

    user_total = len(user_list)
    if network_score > 0 or combined_score > 0: 
        social_media_total = community_size(H.Vertices)
    else: 
        social_media_total = np.ones(len(user_list))

    #get scores to determine order to pick users in different communities 
    for communities, user_count in user_counts.items():
        user_proportion = (user_count / user_total)
        network_proportion = (user_count / social_media_total[communities])
        combined_proportion = (user_count / user_total) * (user_count / social_media_total[communities])

        relevance_score[communities] = user_score * user_proportion + network_score * network_proportion + combined_score * combined_proportion
    
    print(relevance_score)
    sort_score = {k: v for k, v in sorted(relevance_score.items(), key=lambda item: item[1], reverse=True)}
    
    
    #pull users from communtiies that aren't already in the list 
    for community in sort_score.keys():
        for i in community_vertex_list(H.Vertices)[community]:
            if i.value not in user_list: 
                lookalike_list.append(i.value)
                new_users -= 1
                if new_users == 0: 
                    return(lookalike_list)
    
    if new_users > 0: 
        print('Not enough users in network communities to get the desired number of users.')
    return(lookalike_list)

lookalike_audience(user_list, H,2)

{1: 0.5833333333333333, 2: 0.41666666666666663}


['D', 'E']